In [ ]:
"""
Created on Fri Jun 4 15:50 2020

This is a script to convert the NEMO temperature and salinity to potential temperature and practical salinity

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
#from tqdm import tqdm
import gsw
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.melt_functions as meltf

import itertools

import distributed
import glob

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')


In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'isf94' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['ctrl94']:
    yy_start = 1982
    yy_end = 2013
elif nemo_run in ['isf94','isfru94']:
    yy_start = 2014
    yy_end = 2100

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_mask2 = xr.open_mfdataset(inputpath_data+'mask_depth_coord_Ant_stereo.nc')
file_isf = xr.open_mfdataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc', chunks={'x': 533, 'y': 533})
file_isf_cutted = uf.cut_domain_stereo(file_isf, map_lim, map_lim)#.squeeze().drop('time')

#file_TS_orig  = xr.open_mfdataset(inputpath_data+'variables_of_interest_1990_Ant_stereo.nc', chunks={'x': 600, 'y': 600})
file_TS_orig  = xr.open_mfdataset(inputpath_data+'variables_of_interest_'+str(yy_start)+'_Ant_stereo.nc', chunks={'x': 600, 'y': 600})
file_TS_orig_cutted = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim).squeeze().drop('time_counter')

In [ ]:
lon = file_isf_cutted['longitude']
lat = file_isf_cutted['latitude']

In [ ]:
ds_ts

In [ ]:
ts_files = list(sorted(glob.glob(inputpath_data+'variables_of_interest_*_Ant_stereo.nc')))
ds_ts  = xr.open_mfdataset(ts_files, concat_dim='time_counter', combine='nested', chunks={'x': 600, 'y': 600})
ds_ts = ds_ts[['thetao', 'so']]
ds_ts = ds_ts.rename({'time_counter': 'time'})
ds_ts = ds_ts.assign_coords(time=np.arange(yy_start, yy_end+1))
ds_ts_cutted = uf.cut_domain_stereo(ds_ts, map_lim, map_lim)

Prepare the depth axis

In [ ]:
nemo_depth = np.round(file_mask2['gdept_0'].squeeze(dim=['lon','lat']), 3) # round to mm scale - should be enough

Cut out the temperature and salinity and assign the new depth axis

In [ ]:
ds_temp_saline_input = ds_ts_cutted[['thetao', 'so']]
ds_temp_saline_input = ds_temp_saline_input.rename({'thetao': 'temperature', 'so': 'salinity'})
ds_temp_saline_input = ds_temp_saline_input.rename({'deptht': 'depth'})
ds_temp_saline_input['depth'] = np.round(ds_temp_saline_input.depth, 3)
ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values)

In [ ]:
ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values)

In [ ]:
ds_temp_saline_input['salinity'].isel(depth=0,time=0).plot()

In [ ]:
## only points where there is no bedrock - I think not needed for NEMO data
#vert_mask = file_in_T['z_bnds'][:,1] > interp_bed['bed'] 
# only points of open ocean
mask_ocean = ds_temp_saline_input['salinity'].isel(depth=0,time=0) > 0# == 1  #ocean without ice shelf cavity

In [ ]:
mask_ocean.plot()

CONVERT IN-SITU TEMPERATURE FOR OPEN OCEAN REGIONS TO POTENTIAL TEMPERATURE (IF THEY ARE IN-SITU)

In [ ]:
ds_temp_saline_input['theta_ocean'] = xr.apply_ufunc(gsw.pt_from_CT, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['temperature'].where(mask_ocean), dask = 'allowed')
ds_temp_saline_input['salinity_ocean'] = xr.apply_ufunc(gsw.SP_from_SA, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['depth'], lon, lat, dask = 'allowed')
ds_temp_saline_output = ds_temp_saline_input[['theta_ocean', 'salinity_ocean']]

Write the results to multiple files (1 per year)

In [ ]:
yearly_datasets = list(tspf.split_by_chunks(ds_temp_saline_output.unify_chunks(),'time'))
paths = [tspf.create_filepath(ds, 'T_S_theta_ocean_corrected', inputpath_profiles, ds.time[0].values) for ds in yearly_datasets]

this takes approximately 1 min per year

In [ ]:
xr.save_mfdataset(datasets=yearly_datasets, paths=paths)

In [ ]:
inputpath_profiles

In [ ]:
TS_check = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_1980.nc')

In [ ]:
TS_check['theta_ocean'].isel(time=0).isel(depth=50).plot()